# Learning networks

In [1]:
using MLJ, StableRNGs
import DataFrames
@load RidgeRegressor pkg=MultivariateStats

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /home/sandhya/.julia/packages/MLJModels/E8BbE/src/loading.jl:168


import MLJMultivariateStatsInterface

┌ Info: Precompiling MLJMultivariateStatsInterface [1b6a4a23-ba22-4f51-9698-8599985d3728]
└ @ Base loading.jl:1317


 ✔


MLJMultivariateStatsInterface.RidgeRegressor

In [2]:
rng = StableRNG(551234) # for reproducibility

x1 = rand(rng, 300)
x2 = rand(rng, 300)
x3 = rand(rng, 300)
y = exp.(x1 - x2 -2x3 + 0.1*rand(rng, 300))

X = DataFrames.DataFrame(x1=x1, x2=x2, x3=x3)
first(X, 3) |> pretty

┌────────────┬────────────┬────────────┐
│ x1         │ x2         │ x3         │
│ Float64    │ Float64    │ Float64    │
│ Continuous │ Continuous │ Continuous │
├────────────┼────────────┼────────────┤
│ 0.984002   │ 0.771482   │ 0.232099   │
│ 0.891795   │ 0.747399   │ 0.770914   │
│ 0.806395   │ 0.0182751  │ 0.0721645  │
└────────────┴────────────┴────────────┘


In [3]:
test, train = partition(eachindex(y), 0.8);

## Defining a learning network
In MLJ, a learning network is a directed acyclic graph (DAG) whose nodes apply trained or untrained operations such as a predict or transform (trained) or +, vcat etc. (untrained). Learning networks can be seen as pipelines on steroids.

Let's consider the following simple DAG:

Operation DAG

It corresponds to a fairly standard regression workflow: the data is standardized, the target is transformed using a Box-Cox transformation, a ridge regression is applied and the result is converted back by inverting the transform.

Note: actually this DAG is simple enough that it could also have been done with a pipeline.

In [4]:
Xs = source(X)
ys = source(y)

Source @982 ⏎ `AbstractVector{Continuous}`

In [5]:
stand = machine(Standardizer(), Xs)
W = transform(stand, Xs)

Node{Machine{Standardizer,…}} @520
  args:
    1:	Source @348
  formula:
    transform(
        Machine{Standardizer,…} @643, 
        Source @348)

In [6]:
fit!(W, rows=train);

┌ Info: Training Machine{Standardizer,…} @643.
└ @ MLJBase /home/sandhya/.julia/packages/MLJBase/pCCd7/src/machines.jl:342


In [7]:
W()             # transforms all data
W(rows=test, )  # transforms only test data
W(X[3:4, :])    # transforms specific data

,x1,x2,x3
,Float64,Float64,Float64
1,0.856967,-1.59115,-1.48215
2,-1.06436,-1.5056,-0.234452


In [16]:
box_model = UnivariateBoxCoxTransformer()
box = machine(box_model, ys)
z = transform(box, ys)

ridge_model = MLJMultivariateStatsInterface.RidgeRegressor(lambda=0.1)
ridge = machine(ridge_model, W, z)
ẑ = MLJ.predict(ridge, W)

ŷ = inverse_transform(box, ẑ)

Node{Machine{UnivariateBoxCoxTransformer,…}} @876
  args:
    1:	Node{Machine{RidgeRegressor,…}} @426
  formula:
    inverse_transform(
        Machine{UnivariateBoxCoxTransformer,…} @332, 
        predict(
            Machine{RidgeRegressor,…} @589, 
            transform(
                Machine{Standardizer,…} @643, 
                Source @348)))

In [14]:
info("RidgeRegressor", pkg="MultivariateStats").load_path

"MLJMultivariateStatsInterface.RidgeRegressor"

In [17]:
MLJ.fit!(ŷ, rows=train);

┌ Info: Training Machine{UnivariateBoxCoxTransformer,…} @332.
└ @ MLJBase /home/sandhya/.julia/packages/MLJBase/pCCd7/src/machines.jl:342
┌ Info: Not retraining Machine{Standardizer,…} @643. Use `force=true` to force.
└ @ MLJBase /home/sandhya/.julia/packages/MLJBase/pCCd7/src/machines.jl:345
┌ Info: Training Machine{RidgeRegressor,…} @589.
└ @ MLJBase /home/sandhya/.julia/packages/MLJBase/pCCd7/src/machines.jl:342


In [18]:
rms(y[test], ŷ(rows=test))

0.033604963634078514

## Modifying hyperparameters

In [19]:
ridge_model.lambda = 5.0;

In [20]:
MLJ.fit!(ŷ, rows=train)
rms(y[test], ŷ(rows=test))

┌ Info: Not retraining Machine{UnivariateBoxCoxTransformer,…} @332. Use `force=true` to force.
└ @ MLJBase /home/sandhya/.julia/packages/MLJBase/pCCd7/src/machines.jl:345
┌ Info: Not retraining Machine{Standardizer,…} @643. Use `force=true` to force.
└ @ MLJBase /home/sandhya/.julia/packages/MLJBase/pCCd7/src/machines.jl:345
┌ Info: Updating Machine{RidgeRegressor,…} @589.
└ @ MLJBase /home/sandhya/.julia/packages/MLJBase/pCCd7/src/machines.jl:343


0.03834272597361202

## "Arrow" syntax

In [21]:
W = X |> Standardizer()
z = y |> UnivariateBoxCoxTransformer()

Node{Machine{UnivariateBoxCoxTransformer,…}} @388
  args:
    1:	Source @986
  formula:
    transform(
        Machine{UnivariateBoxCoxTransformer,…} @457, 
        Source @986)

In [24]:
ẑ = (W, z) |> MLJMultivariateStatsInterface.RidgeRegressor(lambda=0.1);

In [25]:
ŷ = ẑ |> inverse_transform(z);

In [28]:
MLJ.fit!(ŷ, rows=train)
rms(y[test], ŷ(rows=test))

┌ Info: Training Machine{UnivariateBoxCoxTransformer,…} @457.
└ @ MLJBase /home/sandhya/.julia/packages/MLJBase/pCCd7/src/machines.jl:342
┌ Info: Training Machine{Standardizer,…} @529.
└ @ MLJBase /home/sandhya/.julia/packages/MLJBase/pCCd7/src/machines.jl:342
┌ Info: Training Machine{RidgeRegressor,…} @409.
└ @ MLJBase /home/sandhya/.julia/packages/MLJBase/pCCd7/src/machines.jl:342


0.033604963634078514

In [29]:
ẑ[:lambda] = 5.0;

In [30]:
ẑ.machine.model.lambda = 5.0;

In [31]:
MLJ.fit!(ŷ, rows=train)
rms(y[test], ŷ(rows=test))

┌ Info: Not retraining Machine{UnivariateBoxCoxTransformer,…} @457. Use `force=true` to force.
└ @ MLJBase /home/sandhya/.julia/packages/MLJBase/pCCd7/src/machines.jl:345
┌ Info: Not retraining Machine{Standardizer,…} @529. Use `force=true` to force.
└ @ MLJBase /home/sandhya/.julia/packages/MLJBase/pCCd7/src/machines.jl:345
┌ Info: Updating Machine{RidgeRegressor,…} @409.
└ @ MLJBase /home/sandhya/.julia/packages/MLJBase/pCCd7/src/machines.jl:343


0.03834272597361202

# Learning networks 2

In [1]:
using MLJ
using StableRNGs
import DataFrames


In [2]:
@load RidgeRegressor pkg=MultivariateStats

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /home/sandhya/.julia/packages/MLJModels/E8BbE/src/loading.jl:168


import MLJMultivariateStatsInterface ✔


MLJMultivariateStatsInterface.RidgeRegressor

In [4]:
rng = StableRNG(6616)
x1 = rand(rng, 300)
x2 = rand(rng, 300)
x3 = rand(rng, 300)
y = exp.(x1 - x2 -2x3 + 0.1*rand(rng, 300))
X = DataFrames.DataFrame(x1=x1, x2=x2, x3=x3)

test, train = partition(eachindex(y), 0.8);

In this tutorial we will show how to generate a model from a network; there are two approaches:

* using the @from_network macro
* writing the model in full

### Using the @from_network macro

In [5]:
Xs = source(X)
ys = source(y)

Source @049 ⏎ `AbstractVector{Continuous}`

In [8]:
# First layer

In [6]:
std_model = Standardizer()
stand = machine(std_model, Xs)
W = MLJ.transform(stand, Xs)

Node{Machine{Standardizer,…}} @347
  args:
    1:	Source @124
  formula:
    transform(
        Machine{Standardizer,…} @936, 
        Source @124)

In [7]:
box_model = UnivariateBoxCoxTransformer()
box_mach = machine(box_model, ys)
z = MLJ.transform(box_mach, ys)

Node{Machine{UnivariateBoxCoxTransformer,…}} @801
  args:
    1:	Source @049
  formula:
    transform(
        Machine{UnivariateBoxCoxTransformer,…} @714, 
        Source @049)

In [9]:
# Second Layer

In [13]:
ridge_model = MLJMultivariateStatsInterface.RidgeRegressor(lambda=0.1)
ridge = machine(ridge_model, W, z)
z_pred = predict(ridge, W)

Node{Machine{RidgeRegressor,…}} @056
  args:
    1:	Node{Machine{Standardizer,…}} @347
  formula:
    predict(
        Machine{RidgeRegressor,…} @033, 
        transform(
            Machine{Standardizer,…} @936, 
            Source @124))

In [15]:
y_pred = inverse_transform(box_mach, z_pred)

Node{Machine{UnivariateBoxCoxTransformer,…}} @427
  args:
    1:	Node{Machine{RidgeRegressor,…}} @056
  formula:
    inverse_transform(
        Machine{UnivariateBoxCoxTransformer,…} @714, 
        predict(
            Machine{RidgeRegressor,…} @033, 
            transform(
                Machine{Standardizer,…} @936, 
                Source @124)))

In [17]:
surrogate = Deterministic()
mach = machine(surrogate, Xs, ys; predict=y_pred)

fit!(mach)
predict(mach, X[test[1:5], :])

┌ Info: Training Machine{UnivariateBoxCoxTransformer,…} @714.
└ @ MLJBase /home/sandhya/.julia/packages/MLJBase/pCCd7/src/machines.jl:342
┌ Info: Training Machine{Standardizer,…} @936.
└ @ MLJBase /home/sandhya/.julia/packages/MLJBase/pCCd7/src/machines.jl:342
┌ Info: Training Machine{RidgeRegressor,…} @033.
└ @ MLJBase /home/sandhya/.julia/packages/MLJBase/pCCd7/src/machines.jl:342


5-element Vector{Float64}:
 0.22207406272038532
 0.11459881408622064
 0.5637023411242004
 0.6208523052884072
 0.36914116568152006

In [19]:
@from_network mach begin
    mutable struct CompositeModel
        regressor=ridge_model
    end
end

In [20]:
cm = machine(CompositeModel(), X, y)
res = evaluate!(cm, resampling=Holdout(fraction_train=0.8, rng=51),
                measure=rms)
round(res.measurement[1], sigdigits=3)

0.0136

## Defining a model from scratch
An alternative to the @from_network, is to fully define a new model with its fit method:

In [23]:
mutable struct CompositeModel2 <: DeterministicNetwork
    std_model::Standardizer
    box_model::UnivariateBoxCoxTransformer
    ridge_model::MLJMultivariateStatsInterface.RidgeRegressor
end

function MLJ.fit(m::CompositeModel2, verbosity::Int, X, y)
    Xs = source(X)
    ys = source(y)
    W = MLJ.transform(machine(m.std_model, Xs), Xs)
    box = machine(m.box_model, ys)
    z = MLJ.transform(box, ys)
    ẑ = predict(machine(m.ridge_model, W, z), W)
    ŷ = inverse_transform(box, ẑ)
    mach = machine(Deterministic(), Xs, ys; predict=ŷ)
    fit!(mach, verbosity=verbosity - 1)
    return mach()
end

mdl = CompositeModel2(Standardizer(), UnivariateBoxCoxTransformer(),
                      MLJMultivariateStatsInterface.RidgeRegressor(lambda=0.1))
cm = machine(mdl, X, y)
res = evaluate!(cm, resampling=Holdout(fraction_train=0.8), measure=rms)
round(res.measurement[1], sigdigits=3)

0.0212